In [1]:
import tensorflow as tf
import numpy as np
from music21 import *
import os
import json
import numpy as np
import pickle
import memory_saving_gradients
from random import shuffle
import zipfile
num_notes = 128
length = 19200
noisesize = 19200
learning_rate = 0.001
tf.__dict__["gradients"] = memory_saving_gradients.gradients_memory

music21: Certain music21 functions might need these optional packages: matplotlib, scipy;
                   if you run into errors, install them by following the instructions at
                   http://mit.edu/music21/doc/installing/installAdditional.html


In [2]:
def discriminator(X, num_filters, reuse = None):
    with tf.variable_scope('dis',reuse=reuse):
        layer1 = tf.layers.conv2d(inputs = X, filters = num_filters, kernel_size = [16,num_notes], padding = "same", activation = tf.nn.relu, strides = [1,2])
        layer2 = tf.layers.conv2d(inputs = layer1, filters = num_filters*2, kernel_size = [8,num_notes], padding = "same", activation = tf.nn.relu, strides = [1,2])
        layer3 = tf.layers.conv2d(inputs = layer2, filters = num_filters*4, kernel_size = [4,num_notes], padding = "same", activation = tf.nn.relu, strides = [1,2])
        flat = tf.reshape(layer3, [-1, int(num_notes/8 * length * num_filters*4)])
        dense = tf.layers.dense(inputs=flat, units=128, activation=tf.nn.relu)
        dropout = tf.layers.dropout(inputs=dense, rate=0.4)
        logits = tf.layers.dense(inputs=dropout, units=1)
        output = tf.sigmoid(logits)
        return output, logits
    
        

In [3]:
def generator(z, num_neurons, reuse = None):
    with tf.variable_scope('gen',reuse=reuse):
        cell = tf.contrib.rnn.OutputProjectionWrapper(
        tf.contrib.rnn.BasicLSTMCell(num_units=num_neurons, activation=tf.nn.relu6),
        output_size=num_notes)
        outputs, states = tf.nn.dynamic_rnn(cell, z[:,:,:], dtype=tf.float32)
        outpus = outputs/6
        #lastoutput = outputs[:,-1,:]
        #bias_tensor = tf.fill(tf.shape(lastoutput), .5-a)
        #lastoutput = tf.add(lastoutput, bias_tensor)
        #lastoutput = tf.round(lastoutput)
        #reshapedlastout = tf.reshape(lastoutput,[-1, 1 , num_notes])
        #reshapedlastout = reshapedlastout/6
        #z = tf.concat([z,reshapedlastout],1)
        #for i in range(length-1):
        #    print(i)
        #    outputs, states = tf.nn.dynamic_rnn(cell, z[:,:,-noisesize:], dtype=tf.float32)
        #    lastoutput = outputs[:,-1,:]
        #    reshapedlastout = tf.reshape(lastoutput,[-1,1 , num_notes])
        #    reshapedlastout = reshapedlastout/6
        #    z = tf.concat([z,reshapedlastout],1)
        #z = z[:,-length:,:]
        return tf.reshape(outputs, [-1,19200,128,1])

In [9]:
def reconstruct(array, filename):
    ray = array
    #print(ray.shape)
    #ray = ray.reshape(19200,128)
    ray = ray.reshape(128,19200)
    stream1 = stream.Stream()
    for x in range(ray.shape[1]):
        for y in range(ray.shape[0]):
            if (ray[y,x] == 1) and (ray[y,x-1] == 0):
                w = 0
                while ray[y, x+w] == 1:
                    w += 1
                    if x+w >= 19200:
                        break
                w += 2
                note1 = note.Note(y) #check this
                note1.quarterLength = w/24
                stream1.append(note1)
                note1.offset = x/24 #is this the right way?
    stream1.write("midi", "./outputsreals/" + str(filename) + ".mid")

In [6]:
def next_batch(lis):
    counter = 0
    string = ""
    for item in lis:
        string = str(item)
        zipfilePath = ("./numpys/" + string + ".pickle.zip")
        zip = zipfile.ZipFile(zipfilePath)
        zip.extractall("./numpys")
        pickle_file = open("./numpys/" + string + ".pickle","rb")
        lis = pickle.load(pickle_file)
        pickle_file.close()
        ray = np.array(lis)
        if counter == 0:
            print("made final")
            final = np.reshape(ray, [1,length,num_notes,1])
        else:
            print("catted")
            final = np.concatenate((final, np.reshape(ray, [1,length,num_notes,1])), 0)
            print(final.shape)
        os.remove("./numpys/" + string + ".pickle")
        counter = counter + 1
    return (final)

made final
catted
(2, 19200, 128, 1)
catted
(3, 19200, 128, 1)


In [7]:
with tf.device('/gpu:0'):
    real_music = tf.placeholder(tf.float32, shape = [None, length, num_notes,1])
    z = tf.placeholder(tf.float32,shape=[None,noisesize, num_notes])

In [8]:
with tf.device('/gpu:0'):
    g = generator(z,128)

W0222 21:43:36.130717 140735995515776 deprecation.py:323] From <ipython-input-3-7f582c5828ba>:4: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0222 21:43:36.138009 140735995515776 deprecation.py:323] From <ipython-input-3-7f582c5828ba>:6: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0222 21:43:36.232287 140735995515776 deprecation.py:506] From /anaconda3/envs/mypython3/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1253: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtyp

In [8]:
with tf.device('/gpu:0'):
    D_output_real , D_logits_real = discriminator(real_music, 2)
    D_output_fake, D_logits_fake = discriminator(g,2,reuse=True)

W0221 21:27:57.638404 140735995515776 deprecation.py:323] From <ipython-input-2-a10511fb4723>:3: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.keras.layers.Conv2D instead.
W0221 21:27:57.701711 140735995515776 deprecation.py:323] From <ipython-input-2-a10511fb4723>:7: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0221 21:27:57.725934 140735995515776 deprecation.py:323] From <ipython-input-2-a10511fb4723>:8: dropout (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dropout instead.


In [9]:
def loss_func(logits_in,labels_in):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_in,labels=labels_in))

In [10]:
with tf.device('/gpu:0'):
    D_real_loss = loss_func(D_logits_real,tf.ones_like(D_logits_real)* (0.9))
    D_fake_loss = loss_func(D_logits_fake,tf.zeros_like(D_logits_real))
    D_loss = D_real_loss + D_fake_loss
    G_loss = loss_func(D_logits_fake,tf.ones_like(D_logits_fake))

In [11]:
tvars = tf.trainable_variables()

d_vars = [var for var in tvars if 'dis' in var.name]
g_vars = [var for var in tvars if 'gen' in var.name]

print([v.name for v in d_vars])
print([v.name for v in g_vars])

['dis/conv2d/kernel:0', 'dis/conv2d/bias:0', 'dis/conv2d_1/kernel:0', 'dis/conv2d_1/bias:0', 'dis/conv2d_2/kernel:0', 'dis/conv2d_2/bias:0', 'dis/dense/kernel:0', 'dis/dense/bias:0', 'dis/dense_1/kernel:0', 'dis/dense_1/bias:0']
['gen/rnn/output_projection_wrapper/basic_lstm_cell/kernel:0', 'gen/rnn/output_projection_wrapper/basic_lstm_cell/bias:0', 'gen/rnn/output_projection_wrapper/kernel:0', 'gen/rnn/output_projection_wrapper/bias:0']


In [12]:
with tf.device('/gpu:0'):
    D_trainer = tf.train.AdamOptimizer(learning_rate).minimize(D_loss, var_list=d_vars)
    G_trainer = tf.train.AdamOptimizer(learning_rate).minimize(G_loss, var_list=g_vars)

In [13]:
batch_size = 2
num_songs = 20425
epochs = 100
init = tf.global_variables_initializer()
saver = tf.train.Saver(var_list=g_vars)


In [14]:
tf.summary.scalar("D loss", D_loss)
tf.summary.scalar("G loss", G_loss)
merged_summary_op = tf.summary.merge_all()

In [2]:
with tf.Session() as sess:
    sess.run(init)
    print("inited")
    writer = tf.summary.FileWriter("./realoutput1",sess.graph)
    for e in range(epochs):
        num_batches = num_songs // batch_size
        lis = list(range(num_songs))
        shuffle(lis)
        for i in range(num_batches):
            #print(i)
            batch = next_batch(lis[i*batch_size:i*batch_size + batch_size])
            
            
            batch_z = np.random.uniform(0, 1, size=[batch_size, noisesize, num_notes]) #not sure if 0 or -1
            
            _ = sess.run(D_trainer, feed_dict={real_music: batch, z: batch_z})
            _ = sess.run(G_trainer, feed_dict={z: batch_z})
            summaries = sess.run(merged_summary_op, feed_dict={real_music: batch, z: batch_z})
            writer.add_summary(summaries, e * num_batches + i)
            if i%100 == 0:
                print("100 done!")
                #losd = sess.run(D_loss,feed_dict={real_music: batch_data, z: batch_z})
                #losg = sess.run(G_loss,feed_dict={real_music: batch_data, z: batch_z})
                #print("Generator loss: " + str(losg))
                #print("Discriminatior loss: " + str(losd))
        print("Currently on Epoch {} of {} total...".format(e+1, epochs))
        sample_z = np.random.uniform(0, 1, size=[1, noisesize, num_notes]) 
        gen_sample = sess.run(generator(z, reuse = True),feed_dict={z: sample_z})
        reconstruct(gen_sample, e)
        saver.save(sess, './checkpoints')


NameError: name 'init' is not defined